### **Runnables 구조**

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
# 기본 RAG 구조
vectordb = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectordb.as_retriever()

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the following context:
    {context}  

    Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")  

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | llm  
    | StrOutputParser() 
)

### **Runnables Graph**

In [6]:
# pip install grandalf

# 노드
chain.get_graph().nodes

# 엣지
chain.get_graph().edges

# 그래프 시각화
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

In [10]:
# Chain의 Prompts 확인
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n    {context}  \n\n    Question: {question}'), additional_kwargs={})])]